In [ ]:
#Calculate Alpha and Beta Diversity with Only Bacteria and Fungi
#Read in Taxonomic Data for each Tax ID
TaxID_VAP_MetaData <- read.csv("TaxaData_VAP_MetaDataOnly.csv", header=TRUE, sep=",")
IDSeq_Outputs <- read.csv("IDSeq_VAP_MetaDataOnly.csv", header=TRUE)

levels(TaxID_VAP_MetaData$kingdom)
virus_kingdoms <- c('Bamfordvirae','Orthornavirae','Sangervirae','Shotokuvirae','Viridiplantae')
TaxID_VAP_MetaData$viridae <- grepl("viridae", TaxID_VAP_MetaData$family)
virus_families <- subset(TaxID_VAP_MetaData, TaxID_VAP_MetaData$viridae=="TRUE")
unique(virus_families$family)
fungi_protozoa_kingdoms <- c("Fungi","Metazoa")
;

In [ ]:
#Add the Taxonomic Data to the IDSeq_Outputs
library(dplyr)
IDSeq_Taxa_VAP_MetaData <- inner_join(IDSeq_Outputs,TaxID_VAP_MetaData,by="tax_id")

#Select Viruses
IDSeq_VAP_MetaData_viruses <- subset(IDSeq_Taxa_VAP_MetaData, IDSeq_Taxa_VAP_MetaData$family %in% virus_families)
write.csv(IDSeq_VAP_MetaData_viruses, "IDSeq_VAP_MetaData_viruses.csv", row.names=FALSE, quote=FALSE)

#Select Fungi and Protozoa
IDSeq_VAP_MetaData_fungiprotozoa <- subset(IDSeq_Taxa_VAP_MetaData, IDSeq_Taxa_VAP_MetaData$kingdom %in% fungi_protozoa_kingdoms)
write.csv(IDSeq_VAP_MetaData_fungiprotozoa, "IDSeq_VAP_MetaData_fungiprotozoa.csv", row.names=FALSE, quote=FALSE)

#Exclude Viruses !(y %in% a$x)
`%notin%` <- Negate(`%in%`)
IDSeq_VAP_MetaData_noviruses <- subset(IDSeq_Taxa_VAP_MetaData, IDSeq_Taxa_VAP_MetaData$family %notin% virus_families$family)
unique(IDSeq_VAP_MetaData_noviruses$family)
write.csv(IDSeq_VAP_MetaData_noviruses, "IDSeq_VAP_MetaData_noviruses.csv", row.names=FALSE, quote=FALSE)
;

In [ ]:
#Calculate Alpha and Beta Diversity for Bacteria, Fungi, Protozoa
IDSeq_VAP_noviruses <- read.csv("IDSeq_VAP_MetaData_noviruses.csv", header=TRUE)
IDSeq_Taxa_R <- IDSeq_VAP_noviruses[,c(4,1,6,16,24:30)]

#Reduce the Matrix to Include only the IDSeq_Name, nt_rpm, and genus
IDSeq_Genus <- IDSeq_Taxa_R[,c(2,4,10)]

#Summarize by Genus for Each Sample
IDSeq_Genus_Summ <- IDSeq_Genus %>% group_by(IDSeq_Name,genus) %>% summarise(nt_rpm=sum(nt_rpm))
dim(IDSeq_Genus)
dim(IDSeq_Genus_Summ)
head(IDSeq_Genus_Summ)
write.csv(IDSeq_Genus_Summ,"IDSeq_VAP_byGenus_noviruses.csv", row.names=FALSE, quote=FALSE)
;

In [ ]:
#Take the Genus Data by Sample and Calculate the Alpha Diversity
IDSeq_Genus <- read.csv("IDSeq_VAP_byGenus_noviruses.csv", sep=",",header=TRUE)
head(IDSeq_Genus)

#Load required Library
library(tidyr)

#Make a Matrix of Genus x IDSeq_Name
IDSeq_Genus_Wide <- spread(IDSeq_Genus,genus, nt_rpm)
IDSeq_Genus_Wide[is.na(IDSeq_Genus_Wide)]<-0

#Save the Matrix
write.csv(IDSeq_Genus_Wide,"IDSeq_Wide_VAP_MetaData_noviruses.csv", row.names=FALSE, quote=FALSE)

#Rename Rows as Sample Names
row.names(IDSeq_Genus_Wide) <- IDSeq_Genus_Wide$IDSeq_Name
dim(IDSeq_Genus_Wide)
IDSeq_Genus_Wide<-IDSeq_Genus_Wide[,c(2:1358)]
head(IDSeq_Genus_Wide)
names(IDSeq_Genus_Wide)
;

In [ ]:
#Calculate the Alpha Diversity by Genus 
library(vegan)
Alpha <- diversity(IDSeq_Genus_Wide)
hist(Alpha)
;

In [ ]:
#Summarize the Alpha Diversity by Sample
length(row.names(IDSeq_Genus_Wide))
Alpha_Table <- matrix(data = NA, nrow =  140, ncol =2,dimnames=list(c(1:140),c("IDSeq_Name","Alpha_byGenus")))
Alpha_Table[,c(1)] <- row.names(IDSeq_Genus_Wide)
Alpha_Table[,c(2)] <- Alpha

#Add the Alpha Diversity to the MetaData File
VAP_Samples_MetaData_R <- read.csv("VAP_Samples_MetaData_R.csv", header=TRUE)
VAP_Samples_MetaData_Alpha <- inner_join(VAP_Samples_MetaData_R,Alpha_Table,by="IDSeq_Name",copy=TRUE)
head(VAP_Samples_MetaData_Alpha)
;

In [ ]:
#Calculate the Beta Diversity - BrayCurtis
Beta_BrayCurtis <- vegdist(IDSeq_Genus_Wide, method="bray")
mds_BrayCurtis <- metaMDS(Beta_BrayCurtis)

#Turn Output into DataFrame
mds_BrayCurtis_data <- as.data.frame(mds_BrayCurtis$points)
mds_BrayCurtis_data$IDSeq_Name<-rownames(mds_BrayCurtis_data)
head(mds_BrayCurtis_data)

#Combine MDS1 and MDS2 with MetaData 
VAP_Samples_BrayCurtis <- inner_join(VAP_Samples_MetaData_Alpha, mds_BrayCurtis_data, by="IDSeq_Name")
colnames(VAP_Samples_BrayCurtis)
write.csv(VAP_Samples_BrayCurtis,"VAP_Samples_NoViruses_Diversity.csv", quote=FALSE, row.names=FALSE)
;

In [ ]:
#Calculate Permanova for BrayCurtis
length(row.names(IDSeq_Genus_Wide))
VAP_SampleData <- read.csv("VAP_SampleData.csv", header=TRUE, row.names=1)
VAP_SampleData <- subset(VAP_SampleData, VAP_SampleData$IDSeq_Name %in% row.names(IDSeq_Genus_Wide))
dim(VAP_SampleData)
permanova <- adonis(IDSeq_Genus_Wide ~ VAP_CDC.x + DaysSinceIntubation, data=VAP_SampleData, strata=VAP_SampleData$COMETID, method="bray",perm=999)
permanova
write.csv(permanova$aov.tab,"GenusLevel_NoViruses-BrayCurtisPCOA.csv", quote=F, row.names=TRUE)
;

In [1]:
#Alexandra's Color Scheme
VAP_early <- "#00BFFF"
NoVAP_early <- "#7CCD7C"
VAP_late <- "#FFB90F"
NoVAP_late <- "#EE6A50"

In [ ]:
VAP_Samples_BrayCurtis <- read.csv("VAP_Samples_NoViruses_Diversity.csv", header=TRUE)
#Load libraries
library(ggplot2)
library(dplyr)

#Graph All Timepoints by VAP Status
Graph_VAP_BrayCurtis<-ggplot(VAP_Samples_BrayCurtis,aes(x=MDS1,y=MDS2))+
    geom_point(aes(x=MDS1,y=MDS2,color=VAP_CDC.x,size=5))+
    scale_color_manual(values=c(NoVAP_early,VAP_early))+
    theme(axis.line = element_line(colour = "black"),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        panel.background = element_blank())+
    geom_text(x=-0.25, y=-0.5, label="ANOVA VAP R2=0.024, p=0.015")+
    geom_text(x=-0.25, y=-0.55, label="ANOVA Time R2=0.012, p=0.015")

Graph_VAP_BrayCurtis

ggsave(filename="Graph-BrayCurtisDiversity_noviruses.pdf", plot=Graph_VAP_BrayCurtis,  useDingbats=FALSE, width=8, height=8, units="in")
;

In [ ]:
#Read in Data and Bin By Days since Intubation
VAP_Samples_NoVirus <- read.csv("VAP_Samples_NoViruses_Diversity.csv",header=TRUE)
head(VAP_Samples_NoVirus)

VAP_Samples_NoVirus$DaysSinceIntubation_Binned <- ifelse(VAP_Samples_NoVirus$DaysSinceIntubation<5,"0-4",
                                            ifelse(VAP_Samples_NoVirus$DaysSinceIntubation %in% 5:9,"5-9",
                                            ifelse(VAP_Samples_NoVirus$DaysSinceIntubation %in% 10:14,"10-20","20-35")))

VAP_Samples_NoVirus$DaysSinceIntubation_Binned <- factor(VAP_Samples_NoVirus$DaysSinceIntubation_Binned, levels = c("0-4","5-9","10-20","20-35"))
head(VAP_Samples_NoVirus)
#Summarisse the means for each sample by Bin and Graph
Binned_NoVirus <- VAP_Samples_NoVirus %>% group_by(COMETID,DaysSinceIntubation_Binned) %>% summarise(Alpha_mean = mean(na.omit(Alpha_byGenus)), 
                                                                                            MDS1_mean = mean(na.omit(MDS1)), MDS2_mean=mean(na.omit(MDS2)))
head(Binned_NoVirus)
VAP_byCOMETID <- unique(VAP_Samples_NoVirus[,c(1,8)])
head(VAP_byCOMETID)
Binned_NoVirus <- left_join(Binned_NoVirus, VAP_byCOMETID, by="COMETID")
write.csv(Binned_NoVirus,"NoViruses_Binned_Mean.csv", row.names=FALSE, quote=FALSE)
;

In [ ]:
#Read in Data and Graph BrayCurtis Beta Diversity
Binned_NoVirus <- read.csv("NoViruses_Binned_Mean.csv", header=TRUE)

Binned_NoVirus$DaysSinceIntubation_Binned <- factor(Binned_NoVirus$DaysSinceIntubation_Binned, levels = c("0-4","5-9","10-20","20-35"))

#Anova requires the following assumption
## No extreme outliers 
## Normal Distribution

TwoWayAnova_NoVirus <- lm(MDS1_mean~VAP_CDC.x*DaysSinceIntubation_Binned,data=Binned_NoVirus)
anova(TwoWayAnova_NoVirus)

# Make a graph of the Means with the Bins Side By Side
BrayCurtis_Binned <- ggplot(Binned_NoVirus, aes(x=DaysSinceIntubation_Binned,y=MDS1_mean))+
    geom_boxplot(aes(fill=Binned_NoVirus$VAP_CDC.x),position=position_dodge(1))+
    geom_jitter(aes(fill=Binned_NoVirus$VAP_CDC.x),position=position_dodge(1),size=5)+
    labs(title="VAP")+
        theme(axis.line = element_line(colour = "black"),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        panel.background = element_blank())+
    scale_fill_manual(values=c(NoVAP_early,VAP_early))+
    geom_text(x=1.2, y=0.32, label="ANOVA VAP p=4.1e-5")+
    geom_text(x=1.2, y=0.30, label="ANOVA Time p=0.177")

BrayCurtis_Binned
ggsave(filename="/wynton/group/lynch/Shared/COMET_ER_SRL/Graph-BrayCurtisDiversity_noViruses_Binned.pdf", plot=BrayCurtis_Binned,  useDingbats=FALSE, width=11, height=6, units="in")
ggsave(filename="Graph-BrayCurtisDiversity_noViruses_Binned.pdf", plot=BrayCurtis_Binned,  useDingbats=FALSE, width=11, height=6, units="in")
;

In [ ]:
#Read in Data and Graph BrayCurtis Beta Diversity 
#GRAPH MDS2
Binned_NoVirus <- read.csv("NoViruses_Binned_Mean.csv", header=TRUE)
Binned_NoVirus$DaysSinceIntubation_Binned <- factor(Binned_NoVirus$DaysSinceIntubation_Binned, levels = c("0-4","5-9","10-20","20-35"))

library(ggplot2)
#Anova requires the following assumption
## No extreme outliers 
## Normal Distribution

TwoWayAnova_NoVirus <- lm(MDS2_mean~VAP_CDC.x*DaysSinceIntubation_Binned,data=Binned_NoVirus)
anova(TwoWayAnova_NoVirus)

# Make a graph of the Means with the Bins Side By Side
BrayCurtis_Binned_MDS2 <- ggplot(Binned_NoVirus, aes(x=DaysSinceIntubation_Binned,y=MDS2_mean))+
    geom_boxplot(aes(fill=Binned_NoVirus$VAP_CDC.x),position=position_dodge(1))+
    geom_jitter(aes(fill=Binned_NoVirus$VAP_CDC.x),position=position_dodge(1),size=5)+
    labs(title="VAP")+
        theme(axis.line = element_line(colour = "black"),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        panel.background = element_blank())+
    scale_fill_manual(values=c(NoVAP_early,VAP_early))+
    geom_text(x=1.5, y=0.35, label="ANOVA VAP p=0.034")+
    geom_text(x=1.5, y=0.33, label="ANOVA Time p=0.82")

BrayCurtis_Binned_MDS2
ggsave(filename="/wynton/group/lynch/Shared/COMET_ER_SRL/Graph-BrayCurtisDiversity_MDS2_noViruses_Binned.pdf", plot=BrayCurtis_Binned_MDS2,  useDingbats=FALSE, width=11, height=6, units="in")
ggsave(filename="Graph-BrayCurtisDiversity_MDS2_noViruses_Binned.pdf", plot=BrayCurtis_Binned_MDS2,  useDingbats=FALSE, width=11, height=6, units="in")
;

In [ ]:
#Anova requires the following assumption
Binned_NoVirus <- read.csv("NoViruses_Binned_Mean.csv", header=TRUE)
Binned_NoVirus$DaysSinceIntubation_Binned <- factor(Binned_NoVirus$DaysSinceIntubation_Binned, levels = c("0-4","5-9","10-20","20-35"))

## No extreme outliers - Drop 1158 0-4
#Binned_NoVirus <- Binned_NoVirus[c(1:25,27:44),]
## Normal Distribution


TwoWayAnova_NoVirus <- lm(Alpha_mean~VAP_CDC.x*DaysSinceIntubation_Binned,data=Binned_NoVirus)
anova(TwoWayAnova_NoVirus)

library(ggplot2)
# Make a graph of the Means with the Bins Side By Side
Alpha_Binned <- ggplot(Binned_NoVirus, aes(x=DaysSinceIntubation_Binned,y=Alpha_mean))+
    geom_boxplot(aes(fill=Binned_NoVirus$VAP_CDC.x),position=position_dodge(1))+
    geom_jitter(aes(fill=Binned_NoVirus$VAP_CDC.x),position=position_dodge(1),size=5)+
    labs(title="VAP")+
        theme(axis.line = element_line(colour = "black"),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        panel.border = element_blank(),
        panel.background = element_blank())+
    scale_fill_manual(values=c(NoVAP_early,VAP_early))+
    geom_text(x=3.75, y=3.1, label="ANOVA VAP p=0.0036")+
    geom_text(x=3.75, y=3, label="ANOVA Time p=0.13")

Alpha_Binned
ggsave(filename="/wynton/group/lynch/Shared/COMET_ER_SRL/Graph-AlphaDiversity_noViruses_Binned.pdf", plot=Alpha_Binned,  useDingbats=FALSE, width=11, height=6, units="in")
ggsave(filename="Graph-AlphaDiversity_noViruses_Binned.pdf", plot=Alpha_Binned,  useDingbats=FALSE, width=11, height=6, units="in")
;